In [1]:
!pip install youtube_dl
!pip install transformers
!pip install spacy
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5yawx0rb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5yawx0rb
  Resolved https://github.com/openai/whisper.git to commit 5c1a8c10e762bf9c29fcf6b3e40f17bc8ab09864
  Preparing metadata (setup.py) ... done


In [4]:
# Shout out to Nabil bd !
# https://www.github.com/Nabilphysics
# https://www.nabilbd.com

import youtube_dl
import os

import whisper

In [4]:
# Function to convert youtube link to embed link
# Credit: chatGPT (although chatGPT first mislead me)
def convert_to_embed(link):
    video_id = link.split("=")[-1]
    return 'https://www.youtube.com/embed/' + video_id

# Example usage
# https://www.youtube.com/watch?v=6KUn7AKmC0Y
youtube_link = input("Please enter the Youtube link: ")
embed_link = convert_to_embed(youtube_link)

from IPython.display import HTML
HTML("""
<div align="center">
    <iframe width="560" height="315"
    src=""" + embed_link + """
    </iframe>
</div>
""")

Please enter the Youtube link: https://www.youtube.com/watch?v=AZsmyTE3j9o&t=1s


In [5]:
def download_ytvid_as_mp3():
    #video_url = input("enter url of youtube video:")
    #https://www.youtube.com/watch?v=6KUn7AKmC0Y
    video_info = youtube_dl.YoutubeDL().extract_info(url = youtube_link,download=False)
    # Your file location for Ubuntu
    myfile = "youtubeVideo.mp3"
   
    # If file exists, delete it.
    if os.path.isfile(myfile):
     os.remove(myfile)
    else:
     # If it fails, inform the user.
     print("Error: %s file not found" % myfile)
    
    #filename = f"{video_info['title']}.mp3"
    filename = f"{'youtubeVideo'}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    print("Download complete... {}".format(filename))
download_ytvid_as_mp3()

[youtube] AZsmyTE3j9o: Downloading webpage
Error: youtubeVideo.mp3 file not found
[youtube] AZsmyTE3j9o: Downloading webpage
[download] Destination: youtubeVideo.mp3
[download] 100% of 7.82MiB in 01:41
Download complete... youtubeVideo.mp3


In [5]:
model = whisper.load_model("medium")
#result = model.transcribe('/home/bioforge/Downloads/Spiritual_Authority_1.mp3')
result = model.transcribe('youtubeVideo.mp3')
text = result["text"]
print(text)

 Lexicographers will tell you that time, the word, T-I-M-E, is the most used noun in the English language. We can't get through the day without talking about time all the time. I think about how we use time when we actually talk about it. If you say, meet me at 7 p.m., no one panics. No one says, like, oh my god, what are you talking about with these esoteric concepts about 7 p.m.? You know, we all know what to do operationally. Time, in some sense, is just a label on different events in the universe. The universe happens over and over again at different things we call moments, and time helps us tell the difference between one moment and another. So what time is, I don't think is the problem. The issue, the real puzzles, come about when we talk about the properties that time has. We have a past, we have a present, we have a future. How are they different? Are we moving through it? We have memories of the past, but we have no memories of the future. Why is that? Where does that asymmetr

In [6]:
text = result["text"]
res = len(text.split())
# total no of words
print ("The number of words in string are : " + str(res))

The number of words in string are : 1330


In [7]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
# Credit: https://www.activestate.com/blog/how-to-do-text-summarization-with-python/
#punctuation = punctuation + '\n'
#punctuation

def summarize(text, per):
    nlp = spacy.load('en_core_web_lg')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary 

In [10]:
# extractive summary
extractive_summary = summarize(text, 0.15) # 1.0 = 100% so 0.15 = 15%
print(extractive_summary)

So now we say, if you know that the world is made of atoms, and you know what entropy is in terms of rearranging all those atoms, and you know the past hypothesis, that the entropy of the universe started really low, then you can explain everything that happened after that.And this chain of reasoning goes back 14 billion years to the Big Bang, to the origin of our observable universe in a hot, dense state, a very low entropy state, and the universe has been increasing in entropy ever since.There's a way of talking about human life and entropy, which I think is misguided, which is that we should think about life, literally living, being a biological organism, taking in food and everything, as a fight against increasing entropy.It's the journey from the simple low entropy starting point to the simple high entropy ending point that there's a large space of possibilities where things can be intricate.And there's a natural tendency of things in the universe to go from low entropy to high en

In [8]:
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline

In [11]:
# Pick model
model_name = "google/pegasus-xsum"

# Load pretrained tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Create tokens
tokens = pegasus_tokenizer(extractive_summary, truncation=True, padding="longest", return_tensors="pt")

# Define summarization pipeline 
summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)

In [12]:
# Create summary 
summary = summarizer(extractive_summary, min_length=20, max_length=150)
summary[0]["summary_text"]

'In our series of letters from British scientists, physicist and broadcaster Stephen Hawking explains why the entropy of the universe is increasing.'